In [21]:
from __future__ import print_function
import argparse
from math import log10
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from model import Net
from data import get_training_set, get_test_set

In [22]:
torch.cuda.current_device()

0

In [23]:
torch.cuda.device(0)

In [24]:
torch.cuda.device_count()

1

In [25]:
torch.cuda.get_device_name(0)

'GeForce 920MX'

In [26]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
parser.add_argument('--upscale_factor', type=int, default=3, help="super resolution upscale factor")
parser.add_argument('--batchSize', type=int, default=5, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=100, help='testing batch size')
parser.add_argument('--nEpochs', type=int, default=1000, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.001, help='Learning Rate. Default=0.01')
parser.add_argument('--cuda', action='store_true', help='use cuda?')
parser.add_argument('--threads', type=int, default=5, help='number of threads for data loader to use')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=123, type=<class 'int'>, choices=None, help='random seed to use. Default=123', metavar=None)

In [27]:
opt = parser.parse_args('')

print(opt)

Namespace(batchSize=5, cuda=False, lr=0.001, nEpochs=1000, seed=123, testBatchSize=100, threads=5, upscale_factor=3)


In [28]:
if opt.cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)

device = torch.device("cuda" if opt.cuda else "cpu")

In [29]:
print('===> Loading datasets')
train_set = get_training_set(opt.upscale_factor)
test_set = get_test_set(opt.upscale_factor)
training_data_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)

===> Loading datasets


In [30]:
print('===> Building model')
model = Net(upscale_factor=opt.upscale_factor).to(device)
criterion = nn.MSELoss()

===> Building model


In [31]:
optimizer = optim.Adam(model.parameters(), lr=opt.lr)

In [32]:
def train(epoch):
    epoch_loss = 0
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = batch[0].to(device), batch[1].to(device)

        optimizer.zero_grad()
        loss = criterion(model(input), target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))

In [33]:
def test():
    avg_psnr = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            input, target = batch[0].to(device), batch[1].to(device)

            prediction = model(input)
            mse = criterion(prediction, target)
            psnr = 10 * log10(1 / mse.item())
            avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))

In [34]:
def checkpoint(epoch):
    model_out_path = "model_epoch_{}.pth".format(epoch)
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

In [35]:
for epoch in range(1, opt.nEpochs + 1):
    train(epoch)
    test()
    checkpoint(epoch)

===> Epoch[1](1/40): Loss: 0.4387
===> Epoch[1](2/40): Loss: 0.1745
===> Epoch[1](3/40): Loss: 0.0629
===> Epoch[1](4/40): Loss: 0.0566
===> Epoch[1](5/40): Loss: 0.0483
===> Epoch[1](6/40): Loss: 0.0462
===> Epoch[1](7/40): Loss: 0.0422
===> Epoch[1](8/40): Loss: 0.0332
===> Epoch[1](9/40): Loss: 0.0256
===> Epoch[1](10/40): Loss: 0.0240
===> Epoch[1](11/40): Loss: 0.0337
===> Epoch[1](12/40): Loss: 0.0232
===> Epoch[1](13/40): Loss: 0.0194
===> Epoch[1](14/40): Loss: 0.0387
===> Epoch[1](15/40): Loss: 0.0226
===> Epoch[1](16/40): Loss: 0.0191
===> Epoch[1](17/40): Loss: 0.0157
===> Epoch[1](18/40): Loss: 0.0170
===> Epoch[1](19/40): Loss: 0.0140
===> Epoch[1](20/40): Loss: 0.0214
===> Epoch[1](21/40): Loss: 0.0141
===> Epoch[1](22/40): Loss: 0.0165
===> Epoch[1](23/40): Loss: 0.0094
===> Epoch[1](24/40): Loss: 0.0142
===> Epoch[1](25/40): Loss: 0.0143
===> Epoch[1](26/40): Loss: 0.0118
===> Epoch[1](27/40): Loss: 0.0110
===> Epoch[1](28/40): Loss: 0.0119
===> Epoch[1](29/40): Loss: 0

E:\Anaconda3.6\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Checkpoint saved to model_epoch_1.pth
===> Epoch[2](1/40): Loss: 0.0094
===> Epoch[2](2/40): Loss: 0.0094
===> Epoch[2](3/40): Loss: 0.0124
===> Epoch[2](4/40): Loss: 0.0093
===> Epoch[2](5/40): Loss: 0.0064
===> Epoch[2](6/40): Loss: 0.0116
===> Epoch[2](7/40): Loss: 0.0083
===> Epoch[2](8/40): Loss: 0.0121
===> Epoch[2](9/40): Loss: 0.0082
===> Epoch[2](10/40): Loss: 0.0086
===> Epoch[2](11/40): Loss: 0.0043
===> Epoch[2](12/40): Loss: 0.0074
===> Epoch[2](13/40): Loss: 0.0074
===> Epoch[2](14/40): Loss: 0.0058
===> Epoch[2](15/40): Loss: 0.0088
===> Epoch[2](16/40): Loss: 0.0069
===> Epoch[2](17/40): Loss: 0.0065
===> Epoch[2](18/40): Loss: 0.0087
===> Epoch[2](19/40): Loss: 0.0087
===> Epoch[2](20/40): Loss: 0.0037
===> Epoch[2](21/40): Loss: 0.0074
===> Epoch[2](22/40): Loss: 0.0052
===> Epoch[2](23/40): Loss: 0.0074
===> Epoch[2](24/40): Loss: 0.0063
===> Epoch[2](25/40): Loss: 0.0080
===> Epoch[2](26/40): Loss: 0.0052
===> Epoch[2](27/40): Loss: 0.0077
===> Epoch[2](28/40): Loss

===> Epoch[7](20/40): Loss: 0.0037
===> Epoch[7](21/40): Loss: 0.0051
===> Epoch[7](22/40): Loss: 0.0062
===> Epoch[7](23/40): Loss: 0.0037
===> Epoch[7](24/40): Loss: 0.0047
===> Epoch[7](25/40): Loss: 0.0028
===> Epoch[7](26/40): Loss: 0.0028
===> Epoch[7](27/40): Loss: 0.0036
===> Epoch[7](28/40): Loss: 0.0038
===> Epoch[7](29/40): Loss: 0.0019
===> Epoch[7](30/40): Loss: 0.0061
===> Epoch[7](31/40): Loss: 0.0023
===> Epoch[7](32/40): Loss: 0.0049
===> Epoch[7](33/40): Loss: 0.0033
===> Epoch[7](34/40): Loss: 0.0036
===> Epoch[7](35/40): Loss: 0.0028
===> Epoch[7](36/40): Loss: 0.0015
===> Epoch[7](37/40): Loss: 0.0026
===> Epoch[7](38/40): Loss: 0.0062
===> Epoch[7](39/40): Loss: 0.0028
===> Epoch[7](40/40): Loss: 0.0027
===> Epoch 7 Complete: Avg. Loss: 0.0036
===> Avg. PSNR: 24.0334 dB
Checkpoint saved to model_epoch_7.pth
===> Epoch[8](1/40): Loss: 0.0025
===> Epoch[8](2/40): Loss: 0.0018
===> Epoch[8](3/40): Loss: 0.0034
===> Epoch[8](4/40): Loss: 0.0038
===> Epoch[8](5/40): Lo

===> Epoch[12](36/40): Loss: 0.0036
===> Epoch[12](37/40): Loss: 0.0048
===> Epoch[12](38/40): Loss: 0.0054
===> Epoch[12](39/40): Loss: 0.0029
===> Epoch[12](40/40): Loss: 0.0052
===> Epoch 12 Complete: Avg. Loss: 0.0034
===> Avg. PSNR: 24.2810 dB
Checkpoint saved to model_epoch_12.pth
===> Epoch[13](1/40): Loss: 0.0045
===> Epoch[13](2/40): Loss: 0.0030
===> Epoch[13](3/40): Loss: 0.0026
===> Epoch[13](4/40): Loss: 0.0046
===> Epoch[13](5/40): Loss: 0.0039
===> Epoch[13](6/40): Loss: 0.0029
===> Epoch[13](7/40): Loss: 0.0047
===> Epoch[13](8/40): Loss: 0.0041
===> Epoch[13](9/40): Loss: 0.0019
===> Epoch[13](10/40): Loss: 0.0047
===> Epoch[13](11/40): Loss: 0.0027
===> Epoch[13](12/40): Loss: 0.0044
===> Epoch[13](13/40): Loss: 0.0028
===> Epoch[13](14/40): Loss: 0.0022
===> Epoch[13](15/40): Loss: 0.0036
===> Epoch[13](16/40): Loss: 0.0035
===> Epoch[13](17/40): Loss: 0.0041
===> Epoch[13](18/40): Loss: 0.0034
===> Epoch[13](19/40): Loss: 0.0040
===> Epoch[13](20/40): Loss: 0.0033
=

...

===> Epoch[976](9/40): Loss: 0.0033
===> Epoch[976](10/40): Loss: 0.0017
===> Epoch[976](11/40): Loss: 0.0027
===> Epoch[976](12/40): Loss: 0.0030
===> Epoch[976](13/40): Loss: 0.0020
===> Epoch[976](14/40): Loss: 0.0034
===> Epoch[976](15/40): Loss: 0.0015
===> Epoch[976](16/40): Loss: 0.0012
===> Epoch[976](17/40): Loss: 0.0045
===> Epoch[976](18/40): Loss: 0.0021
===> Epoch[976](19/40): Loss: 0.0020
===> Epoch[976](20/40): Loss: 0.0031
===> Epoch[976](21/40): Loss: 0.0025
===> Epoch[976](22/40): Loss: 0.0015
===> Epoch[976](23/40): Loss: 0.0013
===> Epoch[976](24/40): Loss: 0.0015
===> Epoch[976](25/40): Loss: 0.0026
===> Epoch[976](26/40): Loss: 0.0042
===> Epoch[976](27/40): Loss: 0.0035
===> Epoch[976](28/40): Loss: 0.0017
===> Epoch[976](29/40): Loss: 0.0019
===> Epoch[976](30/40): Loss: 0.0041
===> Epoch[976](31/40): Loss: 0.0028
===> Epoch[976](32/40): Loss: 0.0026
===> Epoch[976](33/40): Loss: 0.0037
===> Epoch[976](34/40): Loss: 0.0021
===> Epoch[976](35/40): Loss: 0.0019
==

===> Epoch[981](16/40): Loss: 0.0018
===> Epoch[981](17/40): Loss: 0.0032
===> Epoch[981](18/40): Loss: 0.0027
===> Epoch[981](19/40): Loss: 0.0027
===> Epoch[981](20/40): Loss: 0.0018
===> Epoch[981](21/40): Loss: 0.0013
===> Epoch[981](22/40): Loss: 0.0026
===> Epoch[981](23/40): Loss: 0.0027
===> Epoch[981](24/40): Loss: 0.0025
===> Epoch[981](25/40): Loss: 0.0035
===> Epoch[981](26/40): Loss: 0.0024
===> Epoch[981](27/40): Loss: 0.0020
===> Epoch[981](28/40): Loss: 0.0036
===> Epoch[981](29/40): Loss: 0.0034
===> Epoch[981](30/40): Loss: 0.0040
===> Epoch[981](31/40): Loss: 0.0030
===> Epoch[981](32/40): Loss: 0.0042
===> Epoch[981](33/40): Loss: 0.0022
===> Epoch[981](34/40): Loss: 0.0024
===> Epoch[981](35/40): Loss: 0.0020
===> Epoch[981](36/40): Loss: 0.0029
===> Epoch[981](37/40): Loss: 0.0032
===> Epoch[981](38/40): Loss: 0.0032
===> Epoch[981](39/40): Loss: 0.0033
===> Epoch[981](40/40): Loss: 0.0034
===> Epoch 981 Complete: Avg. Loss: 0.0026
===> Avg. PSNR: 24.6125 dB
Check

===> Epoch[986](23/40): Loss: 0.0010
===> Epoch[986](24/40): Loss: 0.0034
===> Epoch[986](25/40): Loss: 0.0051
===> Epoch[986](26/40): Loss: 0.0044
===> Epoch[986](27/40): Loss: 0.0016
===> Epoch[986](28/40): Loss: 0.0028
===> Epoch[986](29/40): Loss: 0.0037
===> Epoch[986](30/40): Loss: 0.0028
===> Epoch[986](31/40): Loss: 0.0018
===> Epoch[986](32/40): Loss: 0.0019
===> Epoch[986](33/40): Loss: 0.0022
===> Epoch[986](34/40): Loss: 0.0031
===> Epoch[986](35/40): Loss: 0.0023
===> Epoch[986](36/40): Loss: 0.0022
===> Epoch[986](37/40): Loss: 0.0012
===> Epoch[986](38/40): Loss: 0.0014
===> Epoch[986](39/40): Loss: 0.0029
===> Epoch[986](40/40): Loss: 0.0022
===> Epoch 986 Complete: Avg. Loss: 0.0026
===> Avg. PSNR: 24.6115 dB
Checkpoint saved to model_epoch_986.pth
===> Epoch[987](1/40): Loss: 0.0034
===> Epoch[987](2/40): Loss: 0.0036
===> Epoch[987](3/40): Loss: 0.0019
===> Epoch[987](4/40): Loss: 0.0020
===> Epoch[987](5/40): Loss: 0.0030
===> Epoch[987](6/40): Loss: 0.0025
===> Epo

===> Epoch[991](30/40): Loss: 0.0025
===> Epoch[991](31/40): Loss: 0.0026
===> Epoch[991](32/40): Loss: 0.0021
===> Epoch[991](33/40): Loss: 0.0011
===> Epoch[991](34/40): Loss: 0.0009
===> Epoch[991](35/40): Loss: 0.0025
===> Epoch[991](36/40): Loss: 0.0020
===> Epoch[991](37/40): Loss: 0.0024
===> Epoch[991](38/40): Loss: 0.0015
===> Epoch[991](39/40): Loss: 0.0024
===> Epoch[991](40/40): Loss: 0.0036
===> Epoch 991 Complete: Avg. Loss: 0.0026
===> Avg. PSNR: 24.6171 dB
Checkpoint saved to model_epoch_991.pth
===> Epoch[992](1/40): Loss: 0.0022
===> Epoch[992](2/40): Loss: 0.0023
===> Epoch[992](3/40): Loss: 0.0023
===> Epoch[992](4/40): Loss: 0.0023
===> Epoch[992](5/40): Loss: 0.0031
===> Epoch[992](6/40): Loss: 0.0022
===> Epoch[992](7/40): Loss: 0.0016
===> Epoch[992](8/40): Loss: 0.0020
===> Epoch[992](9/40): Loss: 0.0028
===> Epoch[992](10/40): Loss: 0.0037
===> Epoch[992](11/40): Loss: 0.0042
===> Epoch[992](12/40): Loss: 0.0012
===> Epoch[992](13/40): Loss: 0.0043
===> Epoch[

===> Epoch[996](37/40): Loss: 0.0026
===> Epoch[996](38/40): Loss: 0.0026
===> Epoch[996](39/40): Loss: 0.0027
===> Epoch[996](40/40): Loss: 0.0031
===> Epoch 996 Complete: Avg. Loss: 0.0026
===> Avg. PSNR: 24.6250 dB
Checkpoint saved to model_epoch_996.pth
===> Epoch[997](1/40): Loss: 0.0029
===> Epoch[997](2/40): Loss: 0.0036
===> Epoch[997](3/40): Loss: 0.0022
===> Epoch[997](4/40): Loss: 0.0016
===> Epoch[997](5/40): Loss: 0.0028
===> Epoch[997](6/40): Loss: 0.0039
===> Epoch[997](7/40): Loss: 0.0024
===> Epoch[997](8/40): Loss: 0.0029
===> Epoch[997](9/40): Loss: 0.0020
===> Epoch[997](10/40): Loss: 0.0018
===> Epoch[997](11/40): Loss: 0.0031
===> Epoch[997](12/40): Loss: 0.0026
===> Epoch[997](13/40): Loss: 0.0026
===> Epoch[997](14/40): Loss: 0.0026
===> Epoch[997](15/40): Loss: 0.0033
===> Epoch[997](16/40): Loss: 0.0021
===> Epoch[997](17/40): Loss: 0.0043
===> Epoch[997](18/40): Loss: 0.0024
===> Epoch[997](19/40): Loss: 0.0023
===> Epoch[997](20/40): Loss: 0.0020
===> Epoch[